In [ ]:
# RS_insitu_Comp
# ---
# NaaVRE:
#  cell:
#   inputs:
#    - acolite_processing: List
#   outputs: [] 
#   secrets:
#    - secret_s3_access_key: String
#    - secret_s3_secret_key: String
#   dependencies: 
#    - name: aws.s3
# ...

#    - dummy_output: String

acolite_processing

# Set up the MinIO credentials and endpoint

Sys.setenv(
    "AWS_ACCESS_KEY_ID" = secret_s3_access_key,
    "AWS_SECRET_ACCESS_KEY" = secret_s3_secret_key,
    "AWS_S3_ENDPOINT" = param_s3_server
    )

# Function to download all files from MinIO bucket folder
download_files_from_minio <- function(bucket, folder, local_path) {
  
  # List objects in the folder
  objects <- get_bucket(bucket = bucket, prefix = folder, region="nl-uvalight")
  
  # Iterate through each object and download it
  for (object in objects) {
    file_name <- basename(object$Key)
    local_file_path <- file.path(local_path, file_name)
    cat("Downloading", object$Key, "to", local_file_path, "\n")
    
    # Download the object and save it to the local folder
    save_object(object = object$Key, bucket = bucket, file = local_file_path, region="nl-uvalight")
    
    cat("File", object$Key, "downloaded successfully.\n")
  }
}

# Define your MinIO endpoint and bucket name
bucket_name <- "naa-vre-waddenzee-shared"  # Replace with your bucket name
minio_folder <- "protoDT_WadPP/Input_data/processed_results/"  # Replace with your folder in the bucket
local_folder <- "/tmp/data/processed_results"  # Replace with the local folder path

# Ensure the local folder exists
if (!dir.exists(local_folder)) {
  dir.create(local_folder, recursive = TRUE)
}

# Call the function to download files
download_files_from_minio(bucket = bucket_name, folder = minio_folder, local_path = local_folder)


In [ ]:
# Parameters (DO NOT containerize this cell)
param_s3_server <- "scruffy.lab.uvalight.net:9000"

In [ ]:
# RS_insitu_Comp
# ---
# NaaVRE:
#  cell:
#   inputs:
#    - acolite_processing: List
#   outputs: [] 
#   secrets:
#    - secret_s3_access_key: String
#    - secret_s3_secret_key: String
#   dependencies: 
#    - name: aws.s3
# ...

#    - dummy_output: String

acolite_processing

# Set up the MinIO credentials and endpoint

Sys.setenv(
    "AWS_ACCESS_KEY_ID" = secret_s3_access_key,
    "AWS_SECRET_ACCESS_KEY" = secret_s3_secret_key,
    "AWS_S3_ENDPOINT" = param_s3_server
    )

# Function to download all files from MinIO bucket folder
download_files_from_minio <- function(bucket, folder, local_path) {
  
  # List objects in the folder
  objects <- get_bucket(bucket = bucket, prefix = folder, region="nl-uvalight")
  
  # Iterate through each object and download it
  for (object in objects) {
    file_name <- basename(object$Key)
    local_file_path <- file.path(local_path, file_name)
    cat("Downloading", object$Key, "to", local_file_path, "\n")
    
    # Download the object and save it to the local folder
    save_object(object = object$Key, bucket = bucket, file = local_file_path, region="nl-uvalight")
    
    cat("File", object$Key, "downloaded successfully.\n")
  }
}

# Define your MinIO endpoint and bucket name
bucket_name <- "naa-vre-waddenzee-shared"  # Replace with your bucket name
minio_folder <- "protoDT_WadPP/Input_data/processed_results/"  # Replace with your folder in the bucket
local_folder <- "/tmp/data/processed_results"  # Replace with the local folder path

# Ensure the local folder exists
if (!dir.exists(local_folder)) {
  dir.create(local_folder, recursive = TRUE)
}

# Call the function to download files
download_files_from_minio(bucket = bucket_name, folder = minio_folder, local_path = local_folder)

# # Sentinel data
# station = "DANTZGND"
# filepaths = list.files(local_folder, pattern = station, full.names = TRUE)
# RWS_RS <- lapply(filepaths, function(x) read.csv(x))
# RWS_RS <- do.call("rbind", RWS_RS)
# RWS_RS$time = as.POSIXct(RWS_RS$time, format = "%Y-%m-%d %H:%M:%S")
# RWS_RS

                 
# Combine all data from all stations together
read_acolite_files <- function(station){
    filepaths <- list.files(local_folder, pattern = station, full.names = TRUE)
    files <- lapply(filepaths, function(x) read.csv(x))
    RWS_RS <- do.call("rbind", files)
    RWS_RS$time = as.POSIXct(RWS_RS$time, format = "%Y-%m-%d %H:%M:%S")
    RWS_RS <- RWS_RS[order(RWS_RS$time), ]
    RWS_RS$station = station
    return(RWS_RS)
}

stations <- c("MARSDND", "DOOVWST", "DANTZGND", "VLIESM")
RWS_RS <- lapply(stations, function(x) read_acolite_files(x))
RWS_RS <- do.call("rbind", RWS_RS)

# insitu_data = "tmp/data/RWS_RS.rda"
# save(RWS_RS, file = insitu_data)

# Define your MinIO endpoint and bucket name
bucket_name <- "naa-vre-waddenzee-shared"  # Replace with your bucket name
minio_folder <- "protoDT_WadPP/Input_data/in_situ/"  # Replace with your folder in the bucket
local_folder <- "/tmp/data/in_situ"  # Replace with the local folder path

# Ensure the local folder exists
if (!dir.exists(local_folder)) {
  dir.create(local_folder, recursive = TRUE)
}

# Call the function to download files
download_files_from_minio(bucket = bucket_name, folder = minio_folder, local_path = local_folder)
                 
load(paste0(local_folder,'/RWSbiogeo.rda'))
load(paste0(local_folder,'/RWSstations.rda'))
                 
                 
local_folder <- "/tmp/data/output"  # Replace with the local folder path

# Ensure the local folder exists
if (!dir.exists(local_folder)) {
  dir.create(local_folder, recursive = TRUE)
}

file_name = "chl_validation.png"
file_path = paste0(local_folder, "/", file_name, sep="")

                 
png(file_path, width = 680, height = 580, units = "px", res = 100)
op <- par(mfrow = c(2, 2))
Wad_biogeo_RWS <- subset(RWSbiogeo, 
                        subset=datetime >= "2015-04-20" & datetime < "2021-10-31" & station == 'MARSDND')
plot(Wad_biogeo_RWS$datetime, Wad_biogeo_RWS$Chl, type = "o", pch = 19, col = 'black', ylim = c(0, 30),
     xlab = "", ylab = "Chl (ug/l)", main = "MARSDND")
idx <- which(RWS_RS$station == "MARSDND")
points(RWS_RS$time[idx], RWS_RS$chl_re_gons[idx], type = "p", pch = 19, cex = 1.5, col = "red")

Wad_biogeo_RWS <- subset(RWSbiogeo, 
                        subset=datetime >= "2015-04-20" & datetime < "2021-10-31" & station == 'DOOVBWT')
plot(Wad_biogeo_RWS$datetime, Wad_biogeo_RWS$Chl, type = "o", pch = 19, col = 'black',ylim = c(0, 40),
     xlab = "", ylab = "Chl (ug/l)", main = "DOOVWST")
idx <- which(RWS_RS$station == "DOOVWST")
points(RWS_RS$time[idx], RWS_RS$chl_re_gons[idx], type = "p", pch = 19, cex = 1.5, col = "red")

Wad_biogeo_RWS <- subset(RWSbiogeo, 
                        subset=datetime >= "2015-04-20" & datetime < "2021-10-31" & station == 'DANTZGT')
plot(Wad_biogeo_RWS$datetime, Wad_biogeo_RWS$Chl, type = "o", pch = 19, col = 'black',ylim = c(0, 70),
     xlab = "", ylab = "Chl (ug/l)", main = "DANTZGT")
idx <- which(RWS_RS$station == "DANTZGND")
points(RWS_RS$time[idx], RWS_RS$chl_re_gons[idx], type = "p", pch = 19, cex = 1.5, col = "red")

Wad_biogeo_RWS <- subset(RWSbiogeo, 
                        subset=datetime >= "2015-04-20" & datetime < "2021-10-31" & station == 'VLIESM')
plot(Wad_biogeo_RWS$datetime, Wad_biogeo_RWS$Chl, type = "o", pch = 19, col = 'black',ylim = c(0, 50),
     xlab = "", ylab = "Chl (ug/l)", main = "VLIESM")
idx <- which(RWS_RS$station == "VLIESM")
points(RWS_RS$time[idx], RWS_RS$chl_re_gons[idx], type = "p", pch = 19, cex = 1.5, col = "red")
par(op)
dev.off()

# Upload file to bucket:

miniofile_path = paste0("/protoDT_WadPP/output/",file_name,sep="")
put_object(
    region="nl-uvalight", 
    bucket="naa-vre-waddenzee-shared", 
    file=file_path, 
    object= miniofile_path)
                 
dummy_output = "This one serves as a connection node to the next one"

In [ ]:
# PP_map-R_03
# ---
# NaaVRE:
#  cell:
#   inputs: 
#    - dummy_output: String
#   outputs: []
#   params:
#    - param_s3_server: String
#   secrets:
#    - secret_github_auth_token: String
#    - secret_s3_access_key: String
#    - secret_s3_secret_key: String
#   dependencies:
#    - name: devtools
#    - name: aws.s3
#    - name: ncdf4
#    - name: akima
# ...

dummy_output

# To run as a notebook, open a terminal and install devtools using conda. Running following Commands line by line:
# cd    # this command changes the current directory to your homne directory
# source .bashrc    # this command runs the .bashrc file, which is a script that initializes your shell session. It contains environment variable definitions, aliases and other setup commands
# conda activate lter-life-wadden
# conda install -n lter-life-wadden r-devtools
devtools::install_github("LTER-LIFE/dtR/dtLife",depend=TRUE, force = TRUE, auth_token=secret_github_auth_token)
devtools::install_github("LTER-LIFE/dtR/dtWad", depend=FALSE, force = TRUE, auth_token=secret_github_auth_token)
devtools::install_github("LTER-LIFE/dtR/dtPP", depend=FALSE, force = TRUE, auth_token=secret_github_auth_token)

require(dtLife)
require(dtWad)
require(dtPP)

## Estimate integrated primary production in wadden using acolite-derived chlorophyll a
## and meteorological data from KMNI 

## Open chlorophyll a data already processed by acolite 
library(dtLife)
library(dtWad); library(dtPP)
library(ncdf4)



Sys.setenv(
    "AWS_ACCESS_KEY_ID" = secret_s3_access_key,
    "AWS_SECRET_ACCESS_KEY" = secret_s3_secret_key,
    "AWS_S3_ENDPOINT" = param_s3_server
    )

# Define your MinIO endpoint and bucket name
bucket_name = "naa-vre-waddenzee-shared"  # Replace with your bucket name
file_name = "S2A_MSI_2016_04_11_10_56_07_T31UFV_L2W.nc"
minio_folder = "protoDT_WadPP/Input_data/RS_Download/"  # Replace with your folder in the bucket
miniofile_path = paste0(minio_folder,file_name,sep="")

local_folder = "/tmp/data/Input_data/RS_Download/"  # Replace with the local folder path
local_file_path = paste0(local_folder, file_name)

# download file from minio to local folder
save_object(object = miniofile_path, bucket = bucket_name, file = local_file_path, region="nl-uvalight")

nc = nc_open(local_file_path)
chla = ncvar_get(nc, "chl_re_mishra")

lon = ncvar_get(nc, "lon")
lat = ncvar_get(nc, "lat")
nc_close(nc)


## check the dimension 
dim(chla); dim(lon); dim(lat)

## --------------------------------------------------------------------------
### DOWNSCALE THE DATA TO MAKE ANALYSIS TRACEABLE 
## --------------------------------------------------------------------------
### the dataset is large so we can downscale/block average it to make it run quickly in PP routine 
n <- dim(chla)[1]; m <- dim(chla)[2]
block = 10 # this changes resolution 

chla_block <- lon_block <- lat_block <- matrix(NA, nrow = n %/% block, ncol = m %/% block)

# Downscaling the high resolution data to coarser grid to ease computation 
for(i in 1:(n %/% block)){
  for(j in 1:(m %/% block)){
    iblock <- (1:block) + block*(i - 1)
    jblock <- (1:block) + block*(j - 1)
    lon_block[i, j] <- mean(lon[iblock, jblock])
    lat_block[i, j] <- mean(lat[iblock, jblock])
    chla_block[i, j] <- mean(chla[iblock, jblock])
  }
}

## check dimension and visualize the data
dim(chla_block)
plot3D::image2D(x = lon_block, y = lat_block, z = chla_block, log = "z", asp = 3, main = "block-averaged\non 10 cells")

## --------------------------------------------------------------------------
### DATA PREPARATION FOR PP ESTIMATION 
## --------------------------------------------------------------------------
## Proceed with the main analysis 
ChlLong <- data.frame(longitude   = as.vector(lon_block), 
                      latitude    = as.vector(lat_block), 
                      chlorophyll = as.vector(chla_block))



ChlLong_naomit <- na.omit(ChlLong)

## create mask for region bounded by wadden sea
zz <- mask_xy(ChlLong_naomit, shape = Wad_shape)
zz$chlorophyll[is.na(zz$mask)] <- NA

## visalize the result 
with(na.omit(zz), points2D(x = longitude, y = latitude, colvar = chlorophyll, log = "c", pch = 18, cex = 2, NAcol ="grey", asp = 1.162))

plot(Wad_shape[1], add = TRUE)

### Time period for which we want the PP to be estimated 
time <- as.POSIXct(c("2021-01-23 09:00:00 CET", "2021-01-24 13:00:00 CET", "2021-01-25 09:00:00 CET"))
# time <- as.POSIXct(c("2021-12-23 09:00:00 CET", "2021-12-24 13:00:00 CET", "2021-12-25 09:00:00 CET"))
# 2021-12-25


### Meteorological data from KNMI station
st235   <- subset(Wad_weather, 
                  subset = 
                    station == 235                                 &
                    datetime >= as.POSIXct("2020-12-31 23:59:59")  &
                    datetime <= as.POSIXct("2021-04-01 01:00:10")  )
It.data <- st235[, c("datetime", "radiation")]
It.data$PAR <- It.data$radiation * 0.5
It.data$radiation <- NULL

# Water height and temperature 
# For water height and water temperature  we use the data from station DENHDR and DENHDVSGR
Ht.data  <- Wad_waterheight_HR[ , c("datetime", "DENHDR")]
Ht.data  <- subset(Ht.data, 
                   subset =                                         
                     datetime >= as.POSIXct("2020-12-31 23:59:59")  &
                     datetime <= as.POSIXct("2021-04-01 01:00:10"))

Tt.data  <- Wad_watertemp_HR[ , c("datetime", "DENHDVSGR")]
Tt.data  <- subset(Tt.data, 
                   subset =                                         
                     datetime >= as.POSIXct("2020-12-31 23:59:59")  &
                     datetime <= as.POSIXct("2021-04-01 01:00:10"))

Tmon <- average_timeseries(Tt.data, avgOver="mon", 
                           value = "DENHDVSGR")


### Since I don't have a timeseries of chlorophylla data yet, just concatenate the 
### available data (i.e create a time replica) 
### ****** NOT REALISTIC ********
zz_naomit <- na.omit(zz)
# zz_naomit <- zz_naomit[zz_naomit$longitude > 4.82 & zz_naomit$longitude <= 4.84 & zz_naomit$latitude > 52.96 & zz_naomit$latitude <= 53.00, ]
with(zz_naomit, points2D(x = longitude, y = latitude, colvar = chlorophyll, log = "c", asp = 1))

## duplicate the dataframe to create a spatial timeseries 
zz_long <- data.frame(longitude = c(zz_naomit$longitude, zz_naomit$longitude), 
                      latitude = c(zz_naomit$latitude, zz_naomit$latitude), 
                      chlorophyll = c(zz_naomit$chlorophyll, zz_naomit$chlorophyll), 
                      mask = c(zz_naomit$mask, zz_naomit$mask), 
                      nr = rep(c(1, 2), each = length(zz_naomit$longitude)))
zz_long$date <- as.POSIXct("2021-01-23 09:00:00 CET")
zz_long$date[zz_long$nr == 2] <- as.POSIXct("2021-01-25 09:00:00 CET")

xy_stats <- unique(zz_long[, c("longitude", "latitude")])

### Bathymetric data for computing the volute and depth for which PP will be estimated 
Chl_stats <- map_xy(input.x = as.vector(Wad_depth$longitude), 
                    input.y = as.vector(Wad_depth$latitude), 
                    input.2D = Wad_depth$depth, 
                    output.xy = xy_stats)

Chl_stats$v[Chl_stats$v < 0] <- NA
Chl_stats <- Chl_stats[!(is.na(Chl_stats$v)), ]

points2D(Chl_stats$longitude, Chl_stats$latitude, colvar = Chl_stats$v)


## --------------------------------------------------------------------------
#### Primary production estimation 
## --------------------------------------------------------------------------
temp <- as.data.frame(approx(Tt.data, xout=time)) 
PPall <- NULL


system.time(
  for(i in 1:nrow(Chl_stats)){
    Chl_data <- subset(zz_long, subset = longitude == Chl_stats$longitude[i] & latitude == Chl_stats$latitude[i])
    Chl_data <- Chl_data[, c("date", "chlorophyll")]
    # Chl_data <- Chl_data[!is.na(Chl_data$chlorophyll), ]
    Chl <- as.data.frame(approx(Chl_data, xout = time))
    
    ps <- 13*1.06^(temp$y-20)
    alpha <- 0.049  
    # eopt  <- 4*(150 + 15*ChlRS$Temperature)
    
    PI.par <- data.frame(time = time, 
                         alpha = 0.049*Chl$y,
                         # alpha = 0.049, 
                         eopt = 4*(150 + 15*temp$y), 
                         ps = ps*Chl$y)
    
    
    zmean <- Chl_stats$v[i]
    # cat(zmean)
    
    PP <- integratedPP(zmax = zmean, 
                       times = time, 
                       It.data = It.data, 
                       kz = 0.3, 
                       PI.par = PI.par, 
                       Ht.data= Ht.data, 
                       avgOver = "day")
    
    PPall <- cbind(PPall, PP$ts$PP)
  }
  
)

Haha = "sowhat"


## --------------------------------------------------------------------------
## Visualize the result 
## --------------------------------------------------------------------------
## Check the result 
dim(PPall)

points2D(Chl_stats$longitude, Chl_stats$latitude, colvar = PPall[1, ], xlab = "longitude", ylab = "latitude", clab = "Integrated PP")

## Not sure if the result make absolute sense. Need discussion with Karline 
# op <- par(mfrow = c(2, 2))
# with(na.omit(zz), points2D(x = longitude, y = latitude, colvar = chlorophyll, log = "c", pch = 18, 
#                            cex = 2, NAcol ="grey", asp = 1.162, main = "Chlorophyll data\n landsat-8"))
# plot(WadShape[1], add = TRUE)

# with(zz_naomit, points2D(x = longitude, y = latitude, colvar = chlorophyll, log = "c", asp = 1))

# points2D(Chl_stats$longitude, Chl_stats$latitude, colvar = Chl_stats$z, main = 'Bathymetry')

# points2D(Chl_stats$longitude, Chl_stats$latitude, colvar = PPall[1, ], pch = ".", cex = 2,
#          main = "Integrated PP (mass/surface/time)", xlab = "longitude", 
#          ylab = "latitude", clab = "Integrated PP")


pelagic_pp <- interp(x = Chl_stats$longitude, y = Chl_stats$latitude, z = PPall[1, ], linear = TRUE)

local_folder = "/tmp/data/output/"
file_name = "PP_calculation.png"
file_path = paste0(local_folder, file_name, sep="")

# Ensure the local folder exists
if (!dir.exists(local_folder)) {
  dir.create(local_folder, recursive = TRUE)
}

png(file_path, width = 80, height = 50, units = "cm", res = 100)
with(pelagic_pp, image2D(x = x, y = y, z = z, resfac = 3, asp = 4, 
                  main = "Pelagic Photosynthesis", xlab = "Longitude", 
                  ylab = "Latitude", clab = "mgC/m2/h"))

plot(Wad_shape[1], add = TRUE)

dev.off()
# Upload file to bucket:

miniofile_path = paste0("/protoDT_WadPP/output/",file_name,sep="")
put_object(
    region="nl-uvalight", 
    bucket="naa-vre-waddenzee-shared", 
    file=file_path, 
    object= miniofile_path)

   




In [ ]:
# Insitu-PPJetty-R-4
# ---
# NaaVRE:
#  cell:
#   inputs: []
#   outputs:
#    - Haha: String
#   params:
#    - param_s3_endpoint: String
#   secrets:
#    - secret_github_auth_token: String
#    - secret_s3_access_key: String
#    - secret_s3_secret_key: String
#   dependencies:
#    - name: devtools
#    - name: aws.s3
# ...

Haha = "Dummy"

# To run as a notebook, open a terminal and install devtools using conda. Running following Commands line by line:
# cd    # this command changes the current directory to your homne directory
# source .bashrc    # this command runs the .bashrc file, which is a script that initializes your shell session. It contains environment variable definitions, aliases and other setup commands
# conda activate lter-life-wadden
# conda install -n lter-life-wadden r-devtools
devtools::install_github("LTER-LIFE/dtR/dtLife",depend=TRUE, force = TRUE, auth_token=secret_github_auth_token)
devtools::install_github("LTER-LIFE/dtR/dtWad", depend=FALSE, force = TRUE, auth_token=secret_github_auth_token)
devtools::install_github("LTER-LIFE/dtR/dtPP", depend=FALSE, force = TRUE, auth_token=secret_github_auth_token)

require(dtLife)
require(dtWad)
require(dtPP)

Sys.setenv(
    "AWS_ACCESS_KEY_ID" = secret_s3_access_key,
    "AWS_SECRET_ACCESS_KEY" = secret_s3_secret_key,
    "AWS_S3_ENDPOINT" = param_s3_endpoint
    )

# save data to local folder
save_object(
    region="nl-uvalight", 
    bucket="naa-vre-waddenzee-shared", 
    file="/tmp/data/Jetty.rda", 
    object= "/in_situ/Jetty.rda")

load("/tmp/data/Jetty.rda")
ChlJetty <- subset(Jetty,
subset = datetime >= as.POSIXct("1999-12-15") &
datetime <= as.POSIXct("2021-01-01"))
ChlJetty <- ChlJetty [, c("datetime", "Temperature", "Chl", "Secchi")]
head(ChlJetty)

# Irradiance
# We use the weatherdata from station 235 (DE KOOY VK, 52.9269dgN, 4.7811 dgE) for irradiance.
save_object(
    region="nl-uvalight", 
    bucket="naa-vre-waddenzee-shared", 
    file="/tmp/data/Irradiance.rda", 
    object= "/in_situ/Irradiance.rda")

load("/tmp/data/Irradiance.rda")

# Water height
# Water heights are downloaded from a nearby station, OudeSchild (OUDSD, 4.850192, 53.03884).
save_object(
    region="nl-uvalight", 
    bucket="naa-vre-waddenzee-shared", 
    file="/tmp/data/WHeightJetty.rda", 
    object= "/in_situ/WHeightJetty.rda")

load("/tmp/data/WHeightJetty.rda")

fig_Jetty_tmp_Chl_Irr_WH = "/tmp/data/Jetty_tmp_Chl_Irr_WH.png"

png(fig_Jetty_tmp_Chl_Irr_WH)
par(mfrow=c(2,2))
with(ChlJetty,{
plot(datetime, Chl, type="l")
plot(datetime, Temperature, type="l")
})
plot(Irradiance$datetime, Irradiance$par, type="l")
plot(WHeightJetty$datetime, WHeightJetty$Height, type="l")
dev.off()

# Upload file to bucket:
put_object(
    region="nl-uvalight", 
    bucket="naa-vre-waddenzee-shared", 
    file=fig_Jetty_tmp_Chl_Irr_WH, 
    object="/in_situ/plots/Jetty_tmp_Chl_Irr_WH.png")

# extinction coefficient
# kz varies with time; assume relationship with Secchi:
ChlJetty$kz <- 7/ChlJetty$Secchi # in /m
ChlJetty$kz

# PI parameters
# values for alpha, eopt and ps are not available from the Waddensea
# The values in Brinkman and Jacobs (2023) are used instead (approximately)
# Brinkman & Jacobs, 2023. Gross pelagic primary production in the Ems-Dollard estuary, Journal of Sea
# Research 192 (2023) 102362
# alpha and ps are Chl-specific values -> they need to be multiplied with Chl at each time step
# eopt and ps depend on temperature

# steepness of the light curve, mgC/mgChl/(uEinst/m2/s)
alpha <- 0.049
# optimal light intensity (in uEinst/m2/s - original values in W/m2)
eopt <- 4*(150 + 15*ChlJetty$Temperature)
# assumed temperature dependence for maximal gross production
ps <- 13*1.06^(ChlJetty$Temperature-20) # mgC/mgChl/h
# time-variable PI parameters
PI.par <- data.frame(time = ChlJetty$datetime,
alpha = alpha*ChlJetty$Chl,
eopt = eopt,
ps = ps*ChlJetty$Chl)
head(PI.par)

# Position of the data
datasource <- data.frame(
orig = c("NIOZ", "KNMI", "RWS"),
name = c("Jetty", "De Kooij", "OudeSchild"),
x = c(4.789, 4.7811, 4.850192 ),
y = c(53.002, 52.9269, 53.03884))

fig_Jetty_in_situ_station = "/tmp/data/in_situ_station.png"

png(fig_Jetty_in_situ_station)
plotBathymetry(Marsdiep,
pts = datasource[,c("x", "y")], ptlist = list(cex=4, col= "white"), NAcol="grey", type="image",
main = "Position of the data sets surrounding the NIOZ jetty")
text(datasource$x, datasource$y, labels=1:3, cex=0.7)
legend("bottomright", legend = c(1:3, datasource$name, datasource$orig),
ncol=3, cex=0.7)
dev.off()

# Upload file to bucket:
put_object(
    region="nl-uvalight", 
    bucket="naa-vre-waddenzee-shared", 
    file=fig_Jetty_in_situ_station, 
    object="/in_situ/plots/Jetty_in_situ_station.png")

# Integrated production

# 20 years of data, hourly
times <- seq(from = as.POSIXct("2000-01-01"),
to = as.POSIXct("2020-12-31"),
by = 3600)

# integrated production, averaged over a day
Pprod_day <- integratedPP(
# zn = 10, # water depth
times = times,
convFac = 1,
Ht.data = WHeightJetty, # water height
PI.par = PI.par, # PI parameters
It.data = Irradiance, # Light
kz = ChlJetty[, c("datetime", "kz")],
avgOver = "day",
avgTime = 1)

# plot PP calculation

fig_Jetty_PPs = "/tmp/data/Jetty_PPs.png"

png(fig_Jetty_PPs)
par(mfrow=c(3,3))
plot(Pprod_day, mass="mgC", length="m", time="h")
dev.off()

# Upload file to bucket:
put_object(
    region="nl-uvalight", 
    bucket="naa-vre-waddenzee-shared", 
    file=fig_Jetty_PPs, 
    object="/in_situ/plots/Jetty_PPs.png")